In [16]:
import pandas as pd
from pathlib import Path
import requests
import json
import time
import numpy as np

from globi import formatInteractions


In [17]:
def log_df(df, nrows=5):
    print(df.shape)
    return df.head(nrows)

In [18]:
def print_json(obj):
    print(json.dumps(obj,  indent=4))

In [19]:
raw_data_paths = Path('../data').rglob('taxa_list.csv')
data_paths = [str(path) for path in raw_data_paths]
print(data_paths)

interaction_path = 'outputs/interactions.csv'
de_taxa_path = 'outputs/de_taxa_list.csv'
globi_taxa_path = 'outputs/interactions_taxa_list.csv'

[]


# interactions

## create interactions taxa csv

In [17]:
cols = ['scientific_name', 'common_name', 'taxon_id']
taxa_df = pd.read_csv(de_taxa_path, dtype=str, usecols=cols)
log_df(taxa_df)


(7315, 3)


,taxon_id,common_name,scientific_name
0,1,Animals,Animalia
1,47534,Cnidarians,Cnidaria
2,48921,Hydrozoans,Hydrozoa
3,152823,Siphonophores,Siphonophorae
4,117304,Man O' Wars,Physaliidae


In [18]:
cols = ['subject_taxon_id', 'subject_common_name', 'subject_scientific_name']
interactions_df = pd.read_csv(interaction_path, dtype=str, usecols=cols)
interactions_df.drop_duplicates(inplace=True)

interactions_df.rename(columns = {
    'subject_taxon_id': 'taxon_id', 
    'subject_common_name': 'common_name', 
    'subject_scientific_name': 'scientific_name'
}, inplace=True)

interactions_df['has_globi'] = True

log_df(interactions_df)

(7510, 4)


,taxon_id,common_name,scientific_name,has_globi
0,143452,Lettered Sphinx,Deidamia inscriptum,True
10,47727,maples,Acer,True
18,53178,ribwort plantain,Plantago lanceolata,True
26,60307,Kentucky bluegrass,Poa pratensis,True
47,47124,dicots,Magnoliopsida,True


In [10]:
updated_globi_taxa = taxa_df.merge(interactions_df, how="left" )
log_df(updated_globi_taxa)

(7315, 4)


,taxon_id,common_name,scientific_name,has_globi
0,1,Animals,Animalia,True
1,47534,Cnidarians,Cnidaria,True
2,48921,Hydrozoans,Hydrozoa,True
3,152823,Siphonophores,Siphonophorae,True
4,117304,Man O' Wars,Physaliidae,True


In [11]:
updated_globi_taxa.to_csv(globi_taxa_path, index=False)

## create interactions

In [12]:
cols = ['scientific_name', 'common_name', 'taxon_id']
taxa_df = pd.read_csv(de_taxa_path, dtype=str, usecols=cols)
log_df(taxa_df)

(7315, 3)


,taxon_id,common_name,scientific_name
0,1,Animals,Animalia
1,47534,Cnidarians,Cnidaria
2,48921,Hydrozoans,Hydrozoa
3,152823,Siphonophores,Siphonophorae
4,117304,Man O' Wars,Physaliidae


In [10]:
globi_taxa_df = pd.read_csv(globi_taxa_path, dtype=str)
globi_taxa_df = globi_taxa_df[globi_taxa_df['has_globi'].isna()]
log_df(globi_taxa_df)

(3786, 4)


,taxon_id,common_name,scientific_name,has_globi
6,117302,Portuguese Man o' War,Physalia physalis,NaN
11,986245,Southern Moon Jelly,Aurelia marginalis,NaN
16,83415,Black Jelly,Chrysaora achlyos,NaN
41,370954,Giant Salp,Pyrostremma spinosum,NaN
51,432549,Robson's Blanket Octopus,Tremoctopus robsoni,NaN


In [11]:
interaction_path = 'outputs/interactions.csv'
interaction_df = pd.read_csv(interaction_path, dtype=str)
log_df(interaction_df)

(61213, 7)


,subject_taxon_id,subject_common_name,subject_scientific_name,target_scientific_name,target_common_name,target_taxon_id,interaction
0,143452,Lettered Sphinx,Deidamia inscriptum,Parthenocissus,NaN,50280,eats
1,143452,Lettered Sphinx,Deidamia inscriptum,Vitis,grapevines,60773,eats
2,143452,Lettered Sphinx,Deidamia inscriptum,Ampelopsis brevipedunculata,porcelain berry,457553,eats
3,143452,Lettered Sphinx,Deidamia inscriptum,Vitis vinifera,wine grape,79519,eats
4,143452,Lettered Sphinx,Deidamia inscriptum,Parthenocissus quinquefolia,Virginia creeper,50278,eats


In [12]:
interaction_data = []
count = 0
for index, row in globi_taxa_df.iterrows():
    if count % 10 == 0:
        print(count, end=' ')
        
    for interaction in ['eats', 'eatenBy', 'pollinates', 'pollinatedBy', 'preysOn', 'preyedUponBy']:
        results = formatInteractions(taxa_df, row, interaction, 100)
        if results:
            interaction_data += results
            
    count += 1
            


0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [13]:
new_interactions = pd.DataFrame(interaction_data)
interaction_df = interaction_df.append(new_interactions)
log_df(interaction_df)

(137075, 7)


,subject_taxon_id,subject_common_name,subject_scientific_name,target_scientific_name,target_common_name,target_taxon_id,interaction
0,143452,Lettered Sphinx,Deidamia inscriptum,Parthenocissus,NaN,50280,eats
1,143452,Lettered Sphinx,Deidamia inscriptum,Vitis,grapevines,60773,eats
2,143452,Lettered Sphinx,Deidamia inscriptum,Ampelopsis brevipedunculata,porcelain berry,457553,eats
3,143452,Lettered Sphinx,Deidamia inscriptum,Vitis vinifera,wine grape,79519,eats
4,143452,Lettered Sphinx,Deidamia inscriptum,Parthenocissus quinquefolia,Virginia creeper,50278,eats


create interactions csv

In [14]:
interaction_df.to_csv(interaction_path, index=False)


### add inat common name and taxon id to Globi scientific names

create interactions json

In [10]:

path = '../app/src/lib/data/interactions.csv'
interaction_df.to_csv(path, index = False)